In [6]:
import yaml
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from enum import Enum
import importlib

# Step 1: Lazy Loading Helper Function
def lazy_load(module_name: str, class_name: str):
    try:
        # Dynamically import the module
        module = importlib.import_module(module_name)
        # Get the class from the module
        return getattr(module, class_name)
    except Exception as e:
        raise ValueError(f"Error loading {class_name} from module {module_name}: {e}")

# Step 2: Enum Class for LLM Types
class LLM(str, Enum):
    OPENAI = "openai"
    AZURE_OPENAI = "azure_openai"
    HUGGINGFACE = "huggingface"
    OLLAMA = "ollama"
    COHERE = "cohere"
    VERTEXAI = "vertexai"
    BEDROCK = "bedrock"
    JINA = "jina"
    CUSTOM = "custom"

# Step 3: Map LLM Types to Lazy-loaded Embedding Classes
LLM_MAP = {
    LLM.OPENAI: lazy_load("langchain_openai", "ChatOpenAI"),
    LLM.AZURE_OPENAI: lazy_load("langchain_openai", "AzureChatOpenAI"),
}

# Step 4: Define the LLM Configuration Model
class LLMConfig(BaseModel):
    model_config = {"protected_namespaces": ()}
    
    type: LLM  # Enum to specify the LLM
    model_kwargs: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Model-specific parameters like model name/type")
    custom_class: Optional[str] = None  # Optional: If using a custom class

# Step 5: Load Configuration from YAML
def load_config_from_yaml(file_path: str) -> LLMConfig:
    print("read yaml")
    with open(file_path, "r") as file:
        config_data = yaml.safe_load(file)
    # Convert to LLMConfig object
    return LLMConfig(**config_data["llm"])



In [7]:

# Step 6: Read the Configuration from YAML
config = load_config_from_yaml("config.yaml")

# Step 7: Use the LLM_MAP to Load the Corresponding Model
llm_class = LLM_MAP[config.type]

# Step 8: Instantiate the Model with the Configured Parameters
llm = llm_class(**config.model_kwargs)

# Example of using the instantiated model (assuming it has a `.generate()` method)
response = llm.invoke("What is the capital of France?")
print(response)

read yaml
content='The capital of France is Paris.' response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'

In [9]:
from typing import List, Union
from langchain.pydantic_v1 import Field, BaseModel


class BaseConfig(BaseModel):
    """Base configuration shared across all RAG modules"""
    # input_source: Union[str, List[str]] = Field(..., description="File path, directory path, or URL for input data")
    # test_dataset: str = Field(..., description="Path to CSV file containing test questions")
    
    @classmethod
    def from_yaml(cls, file_path: str) -> 'BaseConfig':
        """Load configuration from a YAML file."""
        with open(file_path, 'r') as file:
            config_dict = yaml.safe_load(file)
        return cls(**config_dict)

    def to_yaml(self, file_path: str) -> None:
        """Save configuration to a YAML file."""
        with open(file_path, 'w') as file:
            yaml.dump(self.model_dump(), file)

# Step 2: Define Pydantic Model for Individual LLM Configuration
class GenerationConfig(BaseConfig):
    type: LLM  # Specifies the LLM type
    model_kwargs: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Model-specific parameters")

# Step 3: Define Pydantic Model for Overall Generation Configuration
class GenerationOptionsConfig(BaseConfig):
    llms: List[GenerationConfig]  # List of LLM configurations

# Step 5: Load YAML File and Parse Configurations
class Generator:
    def __init__(self, config: GenerationOptionsConfig):
        self.llms = []  # List to store instantiated LLMs
        for llm_config in config.llms:
            llm_class = LLM_MAP[llm_config.type]  # Get the corresponding LLM class
config = GenerationOptionsConfig.from_yaml("config.yaml")
gen = Generator(config)


ValidationError: 3 validation errors for GenerationOptionsConfig
input_source
  field required (type=value_error.missing)
test_dataset
  field required (type=value_error.missing)
llms
  field required (type=value_error.missing)